In [1]:
# importing libaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [37]:
# Read in the events data
countryIndi = pd.read_csv('../WEEK 3/Indicators/Country_Indicators.csv')
country = pd.read_csv('../WEEK 3/Indicators/Country.csv')
economicIndi = pd.read_csv('../WEEK 3/Indicators/Economic_Indicators.csv')
enviroIndi= pd.read_csv('../WEEK 3/Indicators/Environmental_Indicators.csv')
qpsdIndi= pd.read_csv('../WEEK 3/Indicators/Public_Debt_Indicators_Agg.csv')
socialIndi= pd.read_csv('../WEEK 3/Indicators/Social_Indicators.csv')
statIndi= pd.read_csv('../WEEK 3/Indicators/Statistical_Indicators.csv')
yearIndi= pd.read_csv('../WEEK 3/Indicators/Year_Table.csv')



In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('../culminating-project-group-2/Database/team2_DB.db')  # Replace with your database file

# Step 1: Retrieve all table names in the database
query_tables = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query_tables, conn)

# Print table names
print("Tables in the database:")
print(tables['name'].tolist())  # Convert DataFrame column to a list




Tables in the database:
['sqlite_sequence', 'Year_Table', 'Country_Year', 'Country_Indicators', 'Economic_Indicators', 'Environmental_Indicators', 'Public_Debt_Indicators_Agg', 'Social_Indicators', 'Statistical_Indicators', 'Country']


In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('../culminating-project-group-2/Database/team2_DB.db')  # Replace with your database file

# Define the SQL query
query = """
    SELECT DISTINCT e."Country Code"
    FROM "Environmental_Indicators" e
    INNER JOIN "Public_Debt_Indicators_Agg" p ON e."Country Code" = p."Country Code"
    INNER JOIN "Economic_Indicators" ec ON e."Country Code" = ec."Country Code"
    INNER JOIN "Social_Indicators" s ON e."Country Code" = s."Country Code"
    INNER JOIN "Statistical_Indicators" si ON e."Country Code" = si."Country_Code"
    INNER JOIN "Country_Indicators" cy ON e."Country Code" = cy."Country Code";
"""

# Execute query and load results into a DataFrame
df = pd.read_sql_query(query, conn)

# Count distinct countries
common_country_count = df["Country Code"].nunique()

# Concatenate country codes into a single string
common_countries = ", ".join(df["Country Code"].unique())

# Print results
print(f"Common Country Count: {common_country_count}")
print(f"Common Countries: {common_countries}")

# Close connection
conn.close()


Common Country Count: 106
Common Countries: ALB, ARG, ARM, AUS, AUT, BGD, BRB, BEL, BOL, BIH, BWA, BRA, BGR, BFA, KHM, CAN, CHL, CHN, COL, COD, CRI, CIV, HRV, CYP, CZE, DNK, DOM, SLV, EST, FJI, FIN, FRA, GEO, DEU, GRC, GTM, HND, HUN, ISL, IDN, IRL, ISR, ITA, JPN, KAZ, KEN, KOR, XKX, KGZ, LVA, LBN, LSO, LTU, LUX, MDG, MWI, MYS, MDV, MLT, MUS, MEX, MDA, MNG, MAR, NPL, NLD, NZL, NIC, NGA, MKD, NOR, PAK, PNG, PRY, PER, PHL, POL, PRT, ROU, RUS, RWA, SAU, SEN, SRB, SYC, SVK, SVN, SLB, ZAF, ESP, LKA, LCA, SUR, SWE, CHE, TJK, TZA, THA, TLS, TGO, TON, TUR, UGA, GBR, USA, URY


In [38]:
# List of DataFrames
dataframes = {'df1': countryIndi, 'df2': country, 'df3': economicIndi, 'df4': enviroIndi, 'df5': qpsdIndi, 'df6': socialIndi, 'df7': statIndi, 'df8': yearIndi}

# Function to calculate overall missing data percentage
def total_missing_percentage(df):
    total_missing = df.isnull().sum().sum()  # Total missing values in DataFrame
    total_values = df.size  # Total number of values in DataFrame
    return (total_missing / total_values) * 100

# Creating a dictionary to store overall missing data percentage
missing_data_summary = {name: total_missing_percentage(df) for name, df in dataframes.items()}

# Converting to DataFrame for better visualization
missing_summary_df = pd.DataFrame(missing_data_summary.items(), columns=['DataFrame', 'Missing Percentage'])

In [42]:
countryIndi.columns

Index(['Country Name', 'Country Code', 'Time', 'Time Code',
       'Adjusted net national income (annual % growth) [NY.ADJ.NNTY.KD.ZG]',
       'Adjusted net national income (constant 2015 US$) [NY.ADJ.NNTY.KD]',
       'Adjusted net national income (current US$) [NY.ADJ.NNTY.CD]',
       'Adjusted net national income per capita (annual % growth) [NY.ADJ.NNTY.PC.KD.ZG]',
       'Adjusted net national income per capita (constant 2015 US$) [NY.ADJ.NNTY.PC.KD]',
       'Adjusted net national income per capita (current US$) [NY.ADJ.NNTY.PC.CD]'],
      dtype='object')

In [39]:
missing_summary_df.head(30)

,DataFrame,Missing Percentage
0,df1,0.654321
1,df2,86.000000
2,df3,0.619002
3,df4,0.600023
4,df5,0.000000
5,df6,0.619344
6,df7,0.000000
7,df8,0.000000
